In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import os
import tensorflow as tf
import keras
import time
import math
import xgboost as xgb
import bayes_opt
%matplotlib inline
os.chdir("C:\\Users\\agastya1208\\Desktop\\Regression")

Using TensorFlow backend.
C:\Users\agastya1208\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [218]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train = train.drop(train.index[1665])
data = pd.concat([train, test], axis=0)

In [219]:
data['desk_id'] = data['desk_id'].fillna('U')
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder().fit(data['desk_id'])
data['desk_id'] = encoder.transform(data['desk_id'])

In [102]:
data1 = data
data1['bought'] = data1['bought'].fillna(np.median(data1['bought']))
#data1['bought'] = data1['bought'].fillna(data1['bought'].median())
#data1['libor_rate'] = data1['libor_rate'].fillna(data1['libor_rate'].median())
data1['libor_rate'] = data1['libor_rate'].fillna(np.median(data1['libor_rate']))
#data1['sold'] = data1['sold'].fillna(data1['sold'].median())
data1['sold'] = data1['sold'].fillna(np.median(data1['sold']))
data1['status'] = data1['status'].fillna(False)
data1['indicator_code'] = data1['indicator_code'].fillna(False)

C:\Users\agastya1208\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4016: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


In [220]:
data1 = data
data1['bought'] = data1['bought'].fillna(data1['bought'].median())
data1['libor_rate'] = data1['libor_rate'].fillna(data1['libor_rate'].median())
data1['sold'] = data1['sold'].fillna(data1['sold'].median())
data1['status'] = data1['status'].fillna(False)
data1['indicator_code'] = data1['indicator_code'].fillna(False)

In [221]:
data1['indicator_code'] = data1['indicator_code'].astype(int)
data1['status'] = data1['status'].astype(int)

In [222]:
GBP_fx = pd.read_excel('C:\\Users\\agastya1208\\Desktop\\Events\\FXTOP_PRICES_GBP_USD.xlsx')
GBP_fx = GBP_fx[['CURRENCY1', 'GBP']].iloc[1:,:]
CHF_fx = pd.read_excel('C:\\Users\\agastya1208\\Desktop\\Events\\FXTOP_PRICES_CHF_USD.xlsx')
CHF_fx = CHF_fx[['CURRENCY1', 'CHF']].iloc[1:,:]
EUR_fx = pd.read_excel('C:\\Users\\agastya1208\\Desktop\\Events\\FXTOP_PRICES_EUR_USD.xlsx')
EUR_fx = EUR_fx[['CURRENCY1', 'EUR']].iloc[1:,:]
JPY_fx = pd.read_excel('C:\\Users\\agastya1208\\Desktop\\Events\\FXTOP_PRICES_JPY_USD.xlsx')
JPY_fx = JPY_fx[['CURRENCY1', 'JPY']].iloc[1:,:]

In [223]:
data1['creation_year'] = data1['creation_date'].astype(str).apply(lambda x: int(x[:4]))
data1['start_year'] = data1['start_date'].astype(str).apply(lambda x: int(x[:4]))
data1['sell_year'] = data1['sell_date'].astype(str).apply(lambda x: int(x[:4]))

In [224]:
creation_date = pd.to_datetime(data1['creation_date'], format='%Y%m%d')
start_date = pd.to_datetime(data1['start_date'], format='%Y%m%d')
sell_date = pd.to_datetime(data1['sell_date'], format='%Y%m%d')

In [225]:
data1['sell_creation_range'] = (sell_date - creation_date).dt.daysp
data1['sell_start_range'] = (sell_date - start_date).dt.days

In [226]:
data1['scheme1'] = (data1['sell_creation_range'] < 0).astype(int)
data1['scheme2'] = (data1['sell_start_range'] < 0).astype(int)
data1['sell_start_range'] = data1['sell_start_range'].apply(lambda x: x if x>0 else 0)

In [227]:
gbp = {}
for i in range(14):
    gbp[GBP_fx.iloc[i,0]] = GBP_fx.iloc[i,1]
chf = {}
for i in range(14):
    chf[CHF_fx.iloc[i,0]] = CHF_fx.iloc[i,1]
eur = {}
for i in range(14):
    eur[EUR_fx.iloc[i,0]] = EUR_fx.iloc[i,1]
jpy = {}
for i in range(14):
    jpy[JPY_fx.iloc[i,0]] = JPY_fx.iloc[i,1]

In [228]:
#Not taking inflation into account
m = data1.shape[0]
final_bought = []
final_sold = []
for i in range(m):
    if data1['currency'].iloc[i] == 'USD':
        final_bought.append(data1['bought'].iloc[i])
        final_sold.append(data1['sold'].iloc[i])
    elif data1['currency'].iloc[i] == 'GBP':
        final_bought.append((data1['bought'].iloc[i])*gbp[data1['start_year'].iloc[i]])
        final_sold.append((data1['sold'].iloc[i])*gbp[data1['sell_year'].iloc[i]])
    elif data1['currency'].iloc[i] == 'EUR':
        final_bought.append((data1['bought'].iloc[i])*eur[data1['start_year'].iloc[i]])
        final_sold.append((data1['sold'].iloc[i])*eur[data1['sell_year'].iloc[i]])
    elif data1['currency'].iloc[i] == 'CHF':
        final_bought.append((data1['bought'].iloc[i])*chf[data1['start_year'].iloc[i]])
        final_sold.append((data1['sold'].iloc[i])*chf[data1['sell_year'].iloc[i]])
    elif data1['currency'].iloc[i] == 'JPY':
        final_bought.append((data1['bought'].iloc[i])*jpy[data1['start_year'].iloc[i]])
        final_sold.append((data1['sold'].iloc[i])*jpy[data1['sell_year'].iloc[i]])

In [229]:
data1['hedge_value'] = data1['hedge_value'].fillna(False)
data1['hedge_value'] = data1['hedge_value'].astype(int)

In [230]:
dummies = pd.get_dummies(data1[['country_code', 'office_id', 'pf_category', 'type']])

In [231]:
_ = data1['libor_rate'].fillna('s')
data1['libor_bank'] = _.apply(lambda x: 0 if x == 's' else 1)

In [232]:
profit = np.array(final_sold) - np.array(final_bought)
bought_and_sold = np.transpose(np.array([np.array(final_bought), np.array(final_sold), 
                                         np.square(final_bought), np.square(final_sold),
                                         np.sqrt(final_bought), np.sqrt(final_sold),
                                         profit, np.square(profit), np.power(profit, 3)]))
profit_array = np.transpose(np.array([profit, np.square(profit), np.power(profit, 3)]))

In [233]:
data1['libor_returns'] = data1['bought']*data1['libor_rate']*data1['sell_start_range']/3000
data1['euribor_returns'] = data1['bought']*data1['euribor_rate']*data1['sell_start_range']/3000

In [234]:
feature = np.concatenate((np.array(dummies),
                          bought_and_sold,
                          np.array(data1[['hedge_value', 'euribor_rate', 'indicator_code',
                                          'sell_creation_range', 'status', 'libor_rate',
                                          'sell_start_range', 'desk_id', 'scheme1',
                                          'scheme2', 'start_year', 'sell_year', 'creation_year']])), axis=1)
train = feature[:9365]
test = feature[9365:]
target = np.array(data1['return'].iloc[:9365])

In [235]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(train, target, test_size=0.3, random_state = 20)

In [192]:
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
grid_model1.fit(xtrain, ytrain)
print("Training Score: ", r2_score(ytrain, grid_model1.predict(xtrain)))
print("Test Score: ", r2_score(ytest, grid_model1.predict(xtest)))

Training Score:  0.999262936862
Test Score:  0.97747527914


In [99]:
grid_model1.fit(train, target)
pred = grid_model1.predict(test)
portfolio_id = data['portfolio_id'][9365:]
test1 = pd.DataFrame({'portfolio_id': portfolio_id, 'return': pred})
test1 = test1[['portfolio_id', 'return']]
test1.to_csv('reg10.csv', index = False)

In [23]:
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
def cv(learning_rate, min_child_weight, max_depth, n_estimators, gamma, max_delta_step,
       colsample_bytree):
    val = np.mean(cross_val_score(XGBRegressor(learning_rate = float(learning_rate), min_child_weight = int(min_child_weight),
                                               max_depth = int(max_depth), n_estimators = int(n_estimators),
                                               gamma = int(gamma), max_delta_step = int(max_delta_step), 
                                               colsample_bytree = float(colsample_bytree),
                                               objective = 'reg:linear'),
                                  train, target, scoring='r2', cv=5))
    return val
gb_params = {'alpha': 1e-5}
bo = BayesianOptimization(cv, {'learning_rate': (0.01, 0.3), 'min_child_weight': (0, 20),
          'max_depth': (5, 20), 'n_estimators': (100, 300),
          'gamma': (0, 5), 'max_delta_step': (0, 10),
          'colsample_bytree': (0.2, 1)})
bo.maximize(init_points=5, n_iter=0, **gb_params, kappa=5)
bo.maximize(n_iter=20, **gb_params, kappa=5)

Initialization
-------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   learning_rate |   max_delta_step |   max_depth |   min_child_weight |   n_estimators | 
    1 | 00m32s |   -2.01022 |             0.9563 |    2.5946 |          0.0422 |           5.5839 |     15.3458 |             4.2491 |       142.0267 | 


KeyboardInterrupt: 

# Neural Network







-------------------------------------------------------------------------------------------------------------------------------

In [273]:
tf.reset_default_graph()
with tf.Session() as sess:
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import mean_squared_error
    scaler = StandardScaler().fit(xtrain)
    xtrain1 = scaler.transform(xtrain)
    xtest1 = scaler.transform(xtest)
    from keras.models import Sequential
    from keras.layers import Dense, Dropout
    from keras.initializers import glorot_uniform
    model = Sequential()
    model.add(Dense(42, input_dim=42, activation='relu', kernel_initializer=glorot_uniform(), bias_initializer='zeros'))
    model.add(Dropout(0.5))
    model.add(Dense(30, activation='relu', kernel_initializer=glorot_uniform(), bias_initializer='zeros'))
    model.add(Dropout(0.5))
    model.add(Dense(20, activation='relu', kernel_initializer=glorot_uniform(), bias_initializer='zeros'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='relu', kernel_initializer=glorot_uniform(), bias_initializer='zeros'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['mse'])
    call = keras.callbacks.TensorBoard(log_dir='C:\\Users\\agastya1208\\Desktop\\Git', histogram_freq=2, 
                                       write_graph=True, write_images=True)
    #model.fit(xtrain1, ytrain, epochs=200, verbose=0, callbacks=[call])
    #print("Training Score: ", r2_score(ytrain, model.predict(xtrain1)))
    #print("Test Score: ", r2_score(ytest, model.predict(xtest1)))
    scaler1 = StandardScaler().fit(train)
    train1 = scaler1.transform(train)
    test1 = scaler1.transform(test)
    model.fit(train1, target, epochs=500, verbose=0,  callbacks=[call])
    pred = model.predict(test1)
    pred = pred.reshape(4801,)
    portfolio_id = data['portfolio_id'][9365:]
    test1 = pd.DataFrame({'portfolio_id': portfolio_id, 'return': pred})
    test1 = test1[['portfolio_id', 'return']]
    test1.to_csv('reg11.csv', index = False)

INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
INFO:tensorflow:Summary name dense_2/kernel:0 is illegal; using dense_2/kernel_0 instead.
INFO:tensorflow:Summary name dense_2/kernel:0 is illegal; using dense_2/kernel_0 instead.
INFO:tensorflow:Summary name dense_2/bias:0 is illegal; using dense_2/bias_0 instead.
INFO:tensorflow:Summary name dense_2/bias:0 is illegal; using dense_2/bias_0 instead.
INFO:tensorflow:Summary name dense_3/kernel:0 is illegal; using dense_3/kernel_0 instead.
INFO:tensorflow:Summary name dense_3/kernel:0 is illegal; using dense_3/kernel_0 instead.
INFO:tensorflow:Summary name dense_3/bias:0 is illegal; using dense_3/bias_0 instead.
INFO:tensorflow:Summary name d

# Random Forest




-------------------------------------------------------------------------------------------------------------------------------

In [ ]:
from sklearn.ensemble import Random

In [37]:
def stacked_predictions(feature, target, models, test):
    m = feature.shape[0]
    n = test.shape[0]
    fold = math.floor(m/11)
    rand = np.random.randint(m, size=m)
    outofbox_predictions = np.zeros((m, 5))
    new_test = np.zeros((n,5))
    for i in range(11):
        indices = list(rand)
        if m%11 == 0:
            holdout_indices = indices[fold*i:fold*(i+1)]
            del indices[fold*i:fold*(i+1)]
        elif m%11 != 0:
            if i<10:
                holdout_indices = indices[fold*i:fold*(i+1)]
                del indices[fold*i:fold*(i+1)]
            elif i == 10:
                holdout_indices = indices[fold*i:]
                del indices[fold*i:]
        training_indices = indices
        holdout = feature[holdout_indices]
        training_feature = feature[training_indices]
        training_target = target[training_indices]
        for j in range(5):
            model = models[j]
            model.fit(training_feature, training_target)
            pred = model.predict(holdout)
            outofbox_predictions[holdout_indices,j] = pred
    for k in range(5):
        model = models[k]
        model.fit(feature, target)
        new_test[:,i] = model.predict(test)
    return outofbox_predictions, new_test